In [1]:
import transformers as trf
import numpy as np
import torch as pt
import pandas as pd
import tqdm
import json

Intel(R) Data Analytics Acceleration Library (Intel(R) DAAL) solvers for sklearn enabled: https://intelpython.github.io/daal4py/sklearn.html


In [2]:
bert = trf.BertForMaskedLM.from_pretrained('bert-base-uncased')
tkr = trf.BertTokenizer.from_pretrained('bert-base-uncased')

In [3]:
data = []
path_to_conceptnet = '../data/ConceptNet/'
with open(f'{path_to_conceptnet}/test.jsonl', 'r') as f:
    for line in f.readlines():
        data.append(json.loads(line))

In [4]:
data[:2]

[{'sub': 'alive',
  'obj': 'think',
  'pred': 'HasSubevent',
  'masked_sentences': ['One of the things you do when you are alive is [MASK].'],
  'obj_label': 'think',
  'uuid': 'd4f11631dde8a43beda613ec845ff7d1'},
 {'sub': 'analyse',
  'obj': 'paralysis',
  'pred': 'HasSubevent',
  'masked_sentences': ['Something that might happen when you analyse something is [MASK].'],
  'obj_label': 'paralysis',
  'uuid': '4b0c94af3a9339e4e5a3f9cbc057fac8'}]

In [5]:
def get_obj(x):
    try:
        return x['obj_label']
    except:
        print(x)
        return None

correct_ans = list(map(get_obj, data))

In [6]:
correct_ans_tks = list(map(lambda x: tkr.convert_tokens_to_ids([x]), correct_ans))

In [7]:
# DATA IS CLEAN
[x for x in correct_ans_tks if len(x) > 1]

[]

In [8]:
correct_ans_tks = [x[0] for x in correct_ans_tks]

In [9]:
data_tks = list(tqdm.tqdm(map(lambda x: tkr.encode(x['masked_sentences'][0]), data)))

29774it [00:07, 4077.22it/s]


In [10]:
data_tks_targets = list(map(lambda x: x.index(tkr.mask_token_id), data_tks))

In [11]:
# ORIGINAL EXAMPLES
# predictions = []
# with pt.no_grad():
#     for tk, target in tqdm.tqdm(zip(data_tks, data_tks_targets)):
#         predictions.append((-bert(pt.tensor(tk).unsqueeze(0))[0][0, target]).argsort())
# predictions

In [12]:
# EXPLORATION
# ORIGINAL ANSWERS FOR FIRST 20 QUESTIONS
predictions = []
with pt.no_grad():
    for tk, target in tqdm.tqdm(zip(data_tks, data_tks_targets[:20])):
        predictions.append((-bert(pt.tensor(tk).unsqueeze(0))[0][0, target]).argsort())
for ranked in predictions:
    print(tkr.convert_ids_to_tokens(ranked[:5]))

20it [00:00, 26.62it/s]

['sleep', 'think', 'cry', 'eat', 'pray']
['wrong', 'dangerous', 'possible', 'happening', 'important']
['dangerous', 'impossible', 'possible', 'important', 'unknown']
['dangerous', 'impossible', 'possible', 'important', 'unknown']
['explanation', 'idea', 'name', 'theory', 'answer']
['worth', 'not', 'doing', 'about', 'beyond']
['cry', 'scream', 'yell', 'talk', 'crying']
['sleep', 'dreams', 'dream', 'dreaming', 'sleeping']
['dangerous', 'bad', 'impossible', 'different', 'normal']
['awake', 'asleep', 'sleep', 'sleeping', 'dead']
['sleep', ',', 'talk', 'rest', 'eat']
['news', 'radio', 'bell', 'call', 'alarm']
['act', 'behave', 'run', 'move', 'look']
['dangerous', 'forbidden', 'impossible', 'safe', 'risky']
['it', 'them', 'that', 'this', 'water']
['dangerous', 'forbidden', 'impossible', 'important', 'safe']
['read', 'opened', 'bought', 'wore', 'dropped']
['dangerous', 'forbidden', 'impossible', 'important', 'safe']
['get', 'go', 'turn', 'walk', 'come']
['taking', 'drawing', 'breathing', 'pum

In [13]:
# EXPLORATION
# REMOVE PERIOD IF IT IS FOUND
predictions = []
with pt.no_grad():
    for tk, target in tqdm.tqdm(zip(data_tks, data_tks_targets[:20])):
        tkm = tk.copy()
        tkm.remove(tkr.convert_tokens_to_ids(['.'])[0])
        predictions.append((-bert(pt.tensor(tkm).unsqueeze(0))[0][0, target]).argsort())
for ranked in predictions:
    print(tkr.convert_ids_to_tokens(ranked[:5]))

20it [00:00, 25.99it/s]

['.', ';', '?', '!', '|']
['.', ';', '?', '!', '...']
['.', ';', '?', '!', '...']
['.', ';', '?', '!', '...']
['.', '?', ';', '!', '...']
['worth', 'not', 'doing', 'about', 'like']
['.', ';', '?', '!', ':']
['sleep', 'dreams', 'dream', 'night', 'home']
['.', ';', '?', '!', '...']
['awake', 'asleep', 'sleeping', 'unconscious', 'dead']
[',', 'be', 'know', 'go', 'die']
['bell', 'radio', 'call', 'news', 'phone']
['run', 'behave', 'walk', 'act', 'move']
['.', ';', '?', '!', '...']
['it', 'them', 'this', 'that', 'water']
['.', ';', '?', '!', '...']
['read', 'in', 'reading', 'from', 'bought']
['.', ';', '?', '!', '...']
['get', 'go', 'turn', 'come', 'walk']
['taking', 'breathing', 'drawing', 'pumping', 'pushing']


In [14]:
# EXPLORATION
# REMOVE SEP IF IT IS FOUND
predictions = []
with pt.no_grad():
    for tk, target in tqdm.tqdm(zip(data_tks, data_tks_targets[:20])):
        tkm = tk.copy()
        tkm.remove(tkr.sep_token_id)
        predictions.append((-bert(pt.tensor(tkm).unsqueeze(0))[0][0, target]).argsort())
for ranked in predictions:
    print(tkr.convert_ids_to_tokens(ranked[:5]))

20it [00:00, 25.33it/s]

['.', '"', 'the', ',', ')']
['.', '"', 'the', 'a', ')']
['.', '"', 'the', 'a', ')']
['.', '"', 'the', 'a', ')']
['one', '.', 'story', 'way', 'start']
['"', '.', 'in', 'on', 'not']
['.', '"', 'the', 'a', ')']
['"', 'i', ',', '.', 'the']
['.', '"', 'the', ')', 'a']
['.', '?', 'are', 'know', 'do']
['be', 'sleep', ',', 'eat', 'say']
['noise', 'voice', 'clock', 'one', 'voices']
['more', '"', ',', '.', 'not']
['.', '"', ')', 'the', "'"]
['it', 'this', 'them', 'one', 'me']
['.', '"', ')', 'me', 'the']
['.', 'in', 'have', 'with', 'are']
['.', '"', ')', 'me', 'the']
['are', 'get', 'live', 'go', 'be']
['taking', 'breathing', 'putting', 'the', 'pumping']


In [15]:
# EXPLORATION
# REPLACE MASK WITH CAT
predictions = []
with pt.no_grad():
    cat_tk_id = tkr.convert_tokens_to_ids(['cat'])[0]
    for tk, target in tqdm.tqdm(zip(data_tks, data_tks_targets[:20])):
        tkm = tk.copy()
        tkm[target] = cat_tk_id
        predictions.append((-bert(pt.tensor(tkm).unsqueeze(0))[0][0, target]).argsort())
for ranked in predictions:
    print(tkr.convert_ids_to_tokens(ranked[:5]))

20it [00:00, 26.89it/s]

['cat', 'cats', 'dog', '##cat', 'kitten']
['cat', 'wrong', 'true', 'different', 'something']
['wrong', 'possible', 'cat', 'not', 'different']
['wrong', 'possible', 'cat', 'not', 'different']
['cat', 'kitten', 'dog', 'thing', 'partner']
['like', 'not', 'in', 'of', 'about']
['cat', 'cats', 'dog', '##cat', 'kitten']
['cat', 'cats', 'kitten', 'kitty', 't']
['cat', 'cats', 'something', 'kitten', 'not']
['cat', 'kitten', 'cats', '##cat', 'bat']
['cat', 'sleep', 'stay', 'eat', 'come']
['cat', 'cats', 'dog', 'kitten', 'thing']
['cat', 'cats', 'kitten', 'not', 'out']
['cat', 'something', 'wrong', 'gone', 'animal']
['cat', 'kitty', 'dog', 'cats', 'kitten']
['cat', 'nice', 'good', 'gone', 'wrong']
['cat', 'in', 'with', 'of', '.']
['cat', 'nice', 'good', 'gone', 'wrong']
['cat', 'get', 'walk', 'go', 'live']
['the', 'cat', 'breathing', 'providing', 'taking']


In [16]:
# STATS
# COUNT ORIGINAL HITS
h1 = 0
h2 = 0
h5 = 0
h10 = 0
h20 = 0
h50 = 0
h100 = 0
total = 0
with pt.no_grad():
    for tk, target, ans in tqdm.tqdm(zip(data_tks, data_tks_targets, correct_ans_tks)):
        tkm = tk.copy()
        total += 1
        ranks = ((-bert(pt.tensor(tkm).unsqueeze(0))[0][0, target]).argsort())
        top_ranks = ranks[:100]
        try:
            idx = list(top_ranks).index(ans)
        except:
            continue
        if 1 > idx:
            h1 += 1
        if 2 > idx:
            h2 += 1
        if 5 > idx:
            h5 += 1
        if 10 > idx:
            h10 += 1
        if 20 > idx:
            h20 += 1
        if 50 > idx:
            h50 += 1
        h100 += 1

29774it [15:16, 32.50it/s]


In [17]:
print(f"""H@1: {h1}
H@2: {h2}
H@5: {h5}
H@10: {h10}
H@20: {h20}
H@50: {h50}
H@100: {h100}
Total: {total}""")

H@1: 3710
H@2: 5285
H@5: 7743
H@10: 9460
H@20: 11219
H@50: 13825
H@100: 15843
Total: 29774


In [18]:
# # STATS: RANKS OF CORRECT ANSWER IN ORIGINAL SAMPLES
# orig_ranks = []
# with pt.no_grad():
#     for tk, target, ans in tqdm.tqdm(zip(data_tks, data_tks_targets, correct_ans_tks[:10])):
#         ranks = ((-bert(pt.tensor(tk).unsqueeze(0))[0][0, target]).argsort())
#         orig_ranks.append(np.where(ranks.numpy() == ans)[0])

In [19]:
# orig_ranks

In [20]:
# STATS
# COUNT HITS WITH DOT REMOVED
h1 = 0
h2 = 0
h5 = 0
h10 = 0
h20 = 0
h50 = 0
h100 = 0
total = 0
with pt.no_grad():
    for tk, target, ans in tqdm.tqdm(zip(data_tks, data_tks_targets, correct_ans_tks)):
        
        tkm = tk.copy()
        try:
            tkm.remove(tkr.convert_tokens_to_ids(['.'])[0])
        except:
            continue
        total += 1
        ranks = ((-bert(pt.tensor(tkm).unsqueeze(0))[0][0, target]).argsort())
        top_ranks = ranks[:100]
        try:
            idx = list(top_ranks).index(ans)
        except:
            continue
        if 1 > idx:
            h1 += 1
        if 2 > idx:
            h2 += 1
        if 5 > idx:
            h5 += 1
        if 10 > idx:
            h10 += 1
        if 20 > idx:
            h20 += 1
        if 50 > idx:
            h50 += 1
        h100 += 1

29774it [14:48, 33.51it/s]


In [21]:
print(f"""H@1: {h1}
H@2: {h2}
H@5: {h5}
H@10: {h10}
H@20: {h20}
H@50: {h50}
H@100: {h100}
Total: {total}""")

H@1: 1977
H@2: 2834
H@5: 3969
H@10: 5430
H@20: 8050
H@50: 11684
H@100: 14011
Total: 29746


In [22]:
# STATS
# COUNT HITS WITH SEP REMOVED
h1 = 0
h2 = 0
h5 = 0
h10 = 0
h20 = 0
h50 = 0
h100 = 0
total = 0
with pt.no_grad():
    for tk, target, ans in tqdm.tqdm(zip(data_tks, data_tks_targets, correct_ans_tks)):
        tkm = tk.copy()
        try:
            tkm.remove(tkr.sep_token_id)
        except:
            continue
        total += 1
        ranks = ((-bert(pt.tensor(tkm).unsqueeze(0))[0][0, target]).argsort())
        top_ranks = ranks[:100]
        try:
            idx = list(top_ranks).index(ans)
        except:
            continue
        if 1 > idx:
            h1 += 1
        if 2 > idx:
            h2 += 1
        if 5 > idx:
            h5 += 1
        if 10 > idx:
            h10 += 1
        if 20 > idx:
            h20 += 1
        if 50 > idx:
            h50 += 1
        h100 += 1

29774it [14:04, 35.26it/s]


In [23]:
print(f"""H@1: {h1}
H@2: {h2}
H@5: {h5}
H@10: {h10}
H@20: {h20}
H@50: {h50}
H@100: {h100}
Total: {total}""")

H@1: 1009
H@2: 1649
H@5: 2612
H@10: 3564
H@20: 4632
H@50: 6399
H@100: 8621
Total: 29774


In [24]:
# STATS
# COUNT HITS WITH MASK REPLACED WITH CAT
h1 = 0
h2 = 0
h5 = 0
h10 = 0
h20 = 0
h50 = 0
h100 = 0
total = 0
with pt.no_grad():
    for tk, target, ans in tqdm.tqdm(zip(data_tks, data_tks_targets, correct_ans_tks)):
        tkm = tk.copy()
        tkm[target] = cat_tk_id
        ranks = ((-bert(pt.tensor(tkm).unsqueeze(0))[0][0, target]).argsort())
        top_ranks = ranks[:100]
        total += 1
        try:
            idx = list(top_ranks).index(ans)
        except:
            continue
        if 1 > idx:
            h1 += 1
            print(tkr.decode(tk))
        if 2 > idx:
            h2 += 1
        if 5 > idx:
            h5 += 1
        if 10 > idx:
            h10 += 1
        if 20 > idx:
            h20 += 1
        if 50 > idx:
            h50 += 1
        h100 += 1

90it [00:02, 33.16it/s]

[CLS] to understand the event " sam went to climb a tree. ", it is important to know that sam could possibly [MASK] and break something. [SEP]


106it [00:03, 34.71it/s]

[CLS] learning a language is for communicating with people who [MASK] that language. [SEP]


118it [00:03, 34.72it/s]

...

29648it [14:12, 34.85it/s]

[CLS] similarity between a trumpet and a rock : are [MASK], not gas or liquid. [SEP]


29720it [14:14, 34.53it/s]

[CLS] cold water is [MASK] and relaxing. [SEP]


29774it [14:15, 34.78it/s]


In [25]:
print(f"""H@1: {h1}
H@2: {h2}
H@5: {h5}
H@10: {h10}
H@20: {h20}
H@50: {h50}
H@100: {h100}
Total: {total}""")

H@1: 702
H@2: 1373
H@5: 2583
H@10: 3566
H@20: 4722
H@50: 6620
H@100: 8470
Total: 29774


In [26]:
# STATS
# COUNT HITS WITH MASK REPLACED WITH RANDOM
h1 = 0
h2 = 0
h5 = 0
h10 = 0
h20 = 0
h50 = 0
h100 = 0
total = 0
with pt.no_grad():
    for tk, target, ans in tqdm.tqdm(zip(data_tks, data_tks_targets, correct_ans_tks)):
        tkm = tk.copy()
        tkm[target] = np.random.choice(tkr.vocab_size)
        ranks = ((-bert(pt.tensor(tkm).unsqueeze(0))[0][0, target]).argsort())
        top_ranks = ranks[:100]
        total += 1
        try:
            idx = list(top_ranks).index(ans)
        except:
            continue
        if 1 > idx:
            h1 += 1
        if 2 > idx:
            h2 += 1
        if 5 > idx:
            h5 += 1
        if 10 > idx:
            h10 += 1
        if 20 > idx:
            h20 += 1
        if 50 > idx:
            h50 += 1
        h100 += 1

29774it [14:19, 34.64it/s]


In [27]:
print(f"""H@1: {h1}
H@2: {h2}
H@5: {h5}
H@10: {h10}
H@20: {h20}
H@50: {h50}
H@100: {h100}
Total: {total}""")

H@1: 1054
H@2: 1722
H@5: 2793
H@10: 3685
H@20: 4749
H@50: 6532
H@100: 8195
Total: 29774


In [28]:
water_tk_id = tkr.convert_tokens_to_ids(['water'])[0]
water_tk_id

2300

In [29]:
# STATS
# COUNT HITS WITH MASK REPLACED WITH WATER
h1 = 0
h2 = 0
h5 = 0
h10 = 0
h20 = 0
h50 = 0
h100 = 0
total = 0
with pt.no_grad():
    for tk, target, ans in tqdm.tqdm(zip(data_tks, data_tks_targets, correct_ans_tks)):
        tkm = tk.copy()
        tkm[target] = water_tk_id
        ranks = ((-bert(pt.tensor(tkm).unsqueeze(0))[0][0, target]).argsort())
        top_ranks = ranks[:100]
        total += 1
        try:
            idx = list(top_ranks).index(ans)
        except:
            continue
        if 1 > idx:
            h1 += 1
        if 2 > idx:
            h2 += 1
        if 5 > idx:
            h5 += 1
        if 10 > idx:
            h10 += 1
        if 20 > idx:
            h20 += 1
        if 50 > idx:
            h50 += 1
        h100 += 1

29774it [14:22, 34.51it/s]


In [30]:
print(f"""H@1: {h1}
H@2: {h2}
H@5: {h5}
H@10: {h10}
H@20: {h20}
H@50: {h50}
H@100: {h100}
Total: {total}""")

H@1: 439
H@2: 1208
H@5: 2445
H@10: 3520
H@20: 4768
H@50: 6819
H@100: 8851
Total: 29774


In [31]:
one_tk_id = tkr.convert_tokens_to_ids(['one'])[0]
one_tk_id

2028

In [32]:
# STATS
# COUNT HITS WITH MASK REPLACED WITH ONE
h1 = 0
h2 = 0
h5 = 0
h10 = 0
h20 = 0
h50 = 0
h100 = 0
total = 0
with pt.no_grad():
    for tk, target, ans in tqdm.tqdm(zip(data_tks, data_tks_targets, correct_ans_tks)):
        tkm = tk.copy()
        tkm[target] = one_tk_id
        ranks = ((-bert(pt.tensor(tkm).unsqueeze(0))[0][0, target]).argsort())
        top_ranks = ranks[:100]
        total += 1
        try:
            idx = list(top_ranks).index(ans)
        except:
            continue
        if 1 > idx:
            h1 += 1
        if 2 > idx:
            h2 += 1
        if 5 > idx:
            h5 += 1
        if 10 > idx:
            h10 += 1
        if 20 > idx:
            h20 += 1
        if 50 > idx:
            h50 += 1
        h100 += 1

29774it [14:22, 34.51it/s]


In [33]:
print(f"""H@1: {h1}
H@2: {h2}
H@5: {h5}
H@10: {h10}
H@20: {h20}
H@50: {h50}
H@100: {h100}
Total: {total}""")

H@1: 530
H@2: 1227
H@5: 2425
H@10: 3427
H@20: 4638
H@50: 6532
H@100: 8359
Total: 29774


In [34]:
dot_tk_id = tkr.convert_tokens_to_ids(['.'])[0]

In [35]:
# STATS
# COUNT HITS WITH LAST DOT REMOVED
h1 = 0
h2 = 0
h5 = 0
h10 = 0
h20 = 0
h50 = 0
h100 = 0
total = 0
with pt.no_grad():
    for tk, target, ans in tqdm.tqdm(zip(data_tks, data_tks_targets, correct_ans_tks)):
        
        tkm = tk.copy()
        if (tkm[-2] == dot_tk_id):
            del(tkm[-2])
        else:
            continue
        total += 1
        ranks = ((-bert(pt.tensor(tkm).unsqueeze(0))[0][0, target]).argsort())
        top_ranks = ranks[:100]
        try:
            idx = list(top_ranks).index(ans)
        except:
            continue
        if 1 > idx:
            h1 += 1
        if 2 > idx:
            h2 += 1
        if 5 > idx:
            h5 += 1
        if 10 > idx:
            h10 += 1
        if 20 > idx:
            h20 += 1
        if 50 > idx:
            h50 += 1
        h100 += 1

29774it [14:04, 35.25it/s]


In [36]:
print(f"""H@1: {h1}
H@2: {h2}
H@5: {h5}
H@10: {h10}
H@20: {h20}
H@50: {h50}
H@100: {h100}
Total: {total}""")

H@1: 2350
H@2: 3327
H@5: 4588
H@10: 6120
H@20: 8798
H@50: 12500
H@100: 14820
Total: 29714


In [37]:
# STATS
# COUNT HITS WITH LAST DOT REMOVED AND MASK IS SECOND TO LAST
h1 = 0
h2 = 0
h5 = 0
h10 = 0
h20 = 0
h50 = 0
h100 = 0
total = 0
with pt.no_grad():
    for tk, target, ans in tqdm.tqdm(zip(data_tks, data_tks_targets, correct_ans_tks)):
        
        tkm = tk.copy()
        if (tkm[-2] == dot_tk_id and tkm[-3] == tkr.mask_token_id):
            del(tkm[-2])
        else:
            continue
        total += 1
        ranks = ((-bert(pt.tensor(tkm).unsqueeze(0))[0][0, target]).argsort())
        top_ranks = ranks[:100]
        try:
            idx = list(top_ranks).index(ans)
        except:
            continue
        if 1 > idx:
            h1 += 1
        if 2 > idx:
            h2 += 1
        if 5 > idx:
            h5 += 1
        if 10 > idx:
            h10 += 1
        if 20 > idx:
            h20 += 1
        if 50 > idx:
            h50 += 1
        h100 += 1

29774it [08:34, 57.87it/s]


In [38]:
print(f"""H@1: {h1}
H@2: {h2}
H@5: {h5}
H@10: {h10}
H@20: {h20}
H@50: {h50}
H@100: {h100}
Total: {total}""")

H@1: 6
H@2: 22
H@5: 111
H@10: 766
H@20: 2606
H@50: 5214
H@100: 6806
Total: 18780


In [39]:
# STATS
# COUNT HITS WITH LAST DOT REMOVED AND MASK IS SECOND TO LAST
h1 = 0
h2 = 0
h5 = 0
h10 = 0
h20 = 0
h50 = 0
h100 = 0
total = 0
with pt.no_grad():
    for tk, target, ans in tqdm.tqdm(zip(data_tks, data_tks_targets, correct_ans_tks)):
        
        tkm = tk.copy()
        if (tkm[-1] == tkr.sep_token_id and tkm[-3] == tkr.mask_token_id):
            del(tkm[-1])
        else:
            continue
        total += 1
        ranks = ((-bert(pt.tensor(tkm).unsqueeze(0))[0][0, target]).argsort())
        top_ranks = ranks[:100]
        try:
            idx = list(top_ranks).index(ans)
        except:
            continue
        if 1 > idx:
            h1 += 1
        if 2 > idx:
            h2 += 1
        if 5 > idx:
            h5 += 1
        if 10 > idx:
            h10 += 1
        if 20 > idx:
            h20 += 1
        if 50 > idx:
            h50 += 1
        h100 += 1

29774it [44:10, 11.23it/s] 


In [40]:
print(f"""H@1: {h1}
H@2: {h2}
H@5: {h5}
H@10: {h10}
H@20: {h20}
H@50: {h50}
H@100: {h100}
Total: {total}""")

H@1: 52
H@2: 175
H@5: 387
H@10: 608
H@20: 922
H@50: 1609
H@100: 2773
Total: 18796


In [41]:
# STATS
# COUNT ORIGINAL HITS
h1 = 0
h2 = 0
h5 = 0
h10 = 0
h20 = 0
h50 = 0
h100 = 0
total = 0
with pt.no_grad():
    for tk, target, ans in tqdm.tqdm(zip(data_tks, data_tks_targets, correct_ans_tks)):
        tkm = tk.copy()
        if (tkm[-3] != tkr.mask_token_id):
            continue
        
        total += 1
        ranks = ((-bert(pt.tensor(tkm).unsqueeze(0))[0][0, target]).argsort())
        top_ranks = ranks[:100]
        try:
            idx = list(top_ranks).index(ans)
        except:
            continue
        if 1 > idx:
            h1 += 1
        if 2 > idx:
            h2 += 1
        if 5 > idx:
            h5 += 1
        if 10 > idx:
            h10 += 1
        if 20 > idx:
            h20 += 1
        if 50 > idx:
            h50 += 1
        h100 += 1

29774it [09:25, 52.64it/s]


In [42]:
print(f"""H@1: {h1}
H@2: {h2}
H@5: {h5}
H@10: {h10}
H@20: {h20}
H@50: {h50}
H@100: {h100}
Total: {total}""")

H@1: 1216
H@2: 1822
H@5: 2934
H@10: 3839
H@20: 4753
H@50: 6387
H@100: 7697
Total: 18796
